In [3]:
import folium
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
from shapely.wkt import loads
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping
from fastkml import kml

In [ ]:
#use this block if data has only GEO_IDs as reference, joins census .csv to county polygon .csv for geo-references
def join_csv_files(file1, file2, key1='GEO_ID' , key2='GEOIDFQ', output_file= 'joined.csv'):
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    merged_df = df1.merge(df2, left_on=key1, right_on=key2, how='inner')
    # Save to a new CSV file
    merged_df.to_csv(output_file, index=False)
    
#the below join should be replaced with the filepath to your tl_2024_us_county data file download, see github wiki
join_csv_files(file1, r"C:\Users\zachh\Downloads\tl_2024_us_county\tl_2024_us_county.csv", key1='GEO_ID', key2='GEOIDFQ', output_file= joined.csv)
print(f"Join complete! Merged file saved as {output_file}")

In [4]:
def convert_to_kml(input_file, output_file='output.kml'):
    gdf = None
    
    # Determine file format and read data accordingly
    if input_file.endswith('.csv'):
        df = pd.read_csv(input_file)
        if 'geometry' not in df.columns:
            raise ValueError("CSV file must contain a 'geometry' column with WKT format.")
        df['geometry'] = df['geometry'].apply(loads)  # Convert WKT to geometry
        gdf = gpd.GeoDataFrame(df, geometry='geometry')
    elif input_file.endswith(('.shp', '.geojson')):
        gdf = gpd.read_file(input_file)
    else:
        raise ValueError("Unsupported file format. Please provide a .csv, .shp, or .geojson file.")
    
    # Ensure gdf is a GeoDataFrame
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Failed to create a valid GeoDataFrame from the input file.")
    
    # Create a KML document
    k = kml.KML()
    k.append(doc)
    
    for _, row in gdf.iterrows():
        if row.geometry is None:
            continue
        placemark = kml.Placemark()
        placemark.geometry = mapping(row.geometry)  # Handles points, lines, and polygons
        placemark.name = str(row.get('name', 'Unnamed'))  # Use 'name' column if available
        doc.append(placemark)
    
    # Write to KML file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(k.to_string(prettyprint=True))
    
    print(f"Conversion complete! KML saved as {output_file}")

In [ ]:
def generate_html(kml_file, output_html='index.html'):
    html_content = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Leaflet KML Map</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css" />
    <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet-omnivore/0.3.4/leaflet-omnivore.min.js"></script>
    <style>
        #map {{ height: 600px; }}
    </style>
</head>
<body>
    <div id="map"></div>
    <script>
        var map = L.map('map').setView([37.7749, -122.4194], 10);
        L.tileLayer('https://tile.openstreetmap.org/{{zoom}}/{{x}}/{{y}}.png', {{
            attribution: '&copy; OpenStreetMap contributors'
        }}).addTo(map);
        var kmlLayer = omnivore.kml('{{kml_file}}').on('ready', function() {{
            map.fitBounds(kmlLayer.getBounds());
        }}).addTo(map);
    </script>
</body>
</html>'''
    with open(output_html, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"HTML file generated: {output_html}")